id: C001

In [2]:
import pandas as pd
import numpy as np
import configparser
import module_table_writer as table_writer
import module_data_wrangling as dw

question_id = 'C001'
#question_title='How were you made aware of the opportunity to do a PhD at the HPI?'
selected_code = 2 #code that indicates that the question was answered

#Enrolled demographic
enrolled_column = 'B003'
enrolled_column_name = 'Enrolled'
enrolled_dictionary = {'0':'Enrolled','B003_01':'Intend to enroll','B003_02':'Intend to re-enroll'}  
#'B003_05':'Do not intend to finish the PhD' -> only two people and answers are mostly empty, so I removed.

#Gender demographic
gender_column = 'B004'
gender_column_name = 'Gender'
gender_dictionary = {1:'Female',2:'Male'}
#3:'Non-binary' no one selected this option
#4:'Diverse' only one person selected this option

#-----------------------------------------------------------------
# LOAD CONFIGURATION FILE
config = configparser.ConfigParser()
config.read('config.ini')
project_path = config['file.loading']['project_path']
data_folder = config['file.loading']['data_folder']
latex_tables_folder = config['file.loading']['latex_tables_folder']
question_index_file = config['file.loading']['question_index_file']
data_file = config['file.loading']['data_file']
sep = config['file.loading']['sep']

#----------------------------------------------------------------
#LOAD QUESTION INDEX (Questions id, Question text, Answer alternatives, Details)
question_index_path = f'{project_path}/{data_folder}/{question_index_file}'
#print(question_index_path)
id_data_frame = pd.read_csv(question_index_path,encoding = 'ISO-8859-1')
filtered_df = id_data_frame[id_data_frame['ID']==question_id]
question_title = filtered_df['Question'].iloc[0]

print('ID:'+question_id+' Question Title:' + question_title)

options_dict = dw.generate_options_dictionary(filtered_df)

#----------------------------------------------------------------
#LOAD DATA FILE AND FILTER NECESSARY COLUMNS
file_name_path = f'{project_path}/{data_folder}/{data_file}'
data_frame = pd.read_csv(file_name_path,encoding = 'ISO-8859-1')

my_list = [enrolled_column, gender_column, question_id]

# Get a list of all keys
options_columns_list = list(options_dict.keys())
options_names_list = list(options_dict.values())

selected_columns = my_list + options_columns_list

# Create a new DataFrame with only the selected columns
data_frame = data_frame[selected_columns]

#RENAMING 
#DataFrame Column with 'Options' instead of Question Text
data_frame = data_frame.rename(columns={data_frame.columns[0]: enrolled_column_name})
data_frame = data_frame.rename(columns={data_frame.columns[1]: gender_column_name})

#Make Program name shorter
#data_frame = data_frame.rename(columns={enrolled_filter: 'enrolled'})
#data_frame['enrolled'] = data_frame['Program'].replace("University of Potsdam,Another University,Not Enrolled')
#data_frame["enrolled"] = data_frame["enrolled"].replace("Other","Others")

#----------------------------------------------------------------
#CREATE LATEX TABLE FILE
tables_path = f'{project_path}/{latex_tables_folder}/{question_id[:2]}/'
tables_file_name = f'{question_id}'

# Set the style for aligning columns to the left
styles = [
    {
        'selector': 'th',
        'props': [('text-align', 'left')]
    },
    {
        'selector': 'td',
        'props': [('text-align', 'left')]
    }
]

# Apply the style to the DataFrame
#df_styled = data_frame.style.set_table_styles(styles)
#display(df_styled)

ID:C001 Question Title:How were you made aware of the opportunity to do a PhD at the HPI?
{'C001_01': 'Web', 'C001_02': 'Newsletters', 'C001_03': 'Academic conferences', 'C001_04': 'Friends/colleagues', 'C001_05': 'I completed my Bachelor/Master at HPI', 'C001_06': 'Other'}


### Enrollment count and percentages

In [8]:
group_column_name = enrolled_column_name
dict_column_names = enrolled_dictionary
#ALL
count_df = dw.percentage_options(df_data=data_frame,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
print('Table for '+'All')
display(count_df)
latex_table = table_writer.write_latex_table(count_df,False, 'All', 
                                    question_id, question_title,
                                    column_format='@{}lcc')
table_writer.table_to_file(latex_table,tables_path,tables_file_name)

# Filter by enrolled
group_categories = dict_column_names.keys()

for group_category in group_categories:
    filtered_df = data_frame[data_frame[group_column_name] == group_category]    
    if(len(filtered_df.index > 0)):#ignores group_categories that are not in teht 
           
        count_df = dw.percentage_options(df_data = filtered_df,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
        group_name = dict_column_names.get(group_category)
        print('Table for '+group_name)
        display(count_df)

        latex_table = table_writer.write_latex_table(count_df,False, group_category, 
                                    question_id, question_title,column_format='@{}lcc')
        table_writer.table_to_file(latex_table,tables_path,tables_file_name)


NameError: name 'All' is not defined

### Gender Count and percentages

In [7]:
group_column_name = gender_column_name
dict_column_names = gender_dictionary
#ALL
count_df = dw.percentage_options(df_data=data_frame,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
print('Table for '+'All')
display(count_df)
latex_table = table_writer.write_latex_table(count_df,False, 'All', 
                                    question_id, question_title,
                                    column_format='@{}lcc')
table_writer.table_to_file(latex_table,tables_path,tables_file_name)

# Filter by enrolled
group_categories = dict_column_names.keys()

for group_category in group_categories:
    filtered_df = data_frame[data_frame[group_column_name] == group_category]    
    if(len(filtered_df.index > 0)):#ignores group_categories that are not in teht 
           
        count_df = dw.percentage_options(df_data = filtered_df,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
        group_name = dict_column_names.get(group_category)
        print('Table for '+group_name)
        display(count_df)

        latex_table = table_writer.write_latex_table(count_df,False, group_category, 
                                    question_id, question_title,column_format='@{}lcc')
        table_writer.table_to_file(latex_table,tables_path,tables_file_name)


,Option,Count,Percentage
0,Web,16,14.04
1,Newsletters,2,1.75
2,Academic conferences,4,3.51
3,Friends/colleagues,38,33.33
4,I completed my Bachelor/Master at HPI,45,39.47
5,Other,9,7.89
6,TOTAL,114,100.00


Table for Female


,Option,Count,Percentage
0,Web,4,16.67
1,Newsletters,2,8.33
2,Academic conferences,2,8.33
3,Friends/colleagues,10,41.67
4,I completed my Bachelor/Master at HPI,5,20.83
5,Other,1,4.17
6,TOTAL,24,100.00


Table for Male


,Option,Count,Percentage
0,Web,12,13.48
1,Newsletters,0,0.00
2,Academic conferences,2,2.25
3,Friends/colleagues,28,31.46
4,I completed my Bachelor/Master at HPI,39,43.82
5,Other,8,8.99
6,TOTAL,89,100.00


### Free text answers

In [ ]:
df_text = pd.DataFrame(data_frame[['Comments','enrolled']]).dropna()
df_text = df_text.sort_values(by=['enrolled'])
#df_styled = df_text.style.set_table_styles(styles)

table_caption=f'Comments for {question_id}-{question_title}'
label_name = f'{question_id}-FreeText_table'
latex_table = df_text.to_latex(index=False,
                               caption=table_caption, 
                               label=label_name,
                               column_format='@{}p{0.65\\textwidth}p{0.35\\textwidth}')

table_writer.table_to_file(latex_table,tables_path,tables_file_name)


### Making sense of the data

#### Surprising findings:
- Number of recommendations from friends and colleagues (maybe a program to incentivize that?)
- Web pages or search amounts to more than 34%, which is also surprising.
- Low proportion on directed/planned initiatives like conferences and magazines.